# Getting all arrivals to all stops of a given line in a given day

## Install dependencies

This notebook requires two dependencies which can be installed with the following command `pip install pandas open-bus-stride-client`.

You can also launch it online at [this URL](https://mybinder.org/v2/gh/hasadna/open-bus-stride-client/HEAD?labpath=notebooks%2Fgetting%20all%20arrivals%20to%20all%20stops%20of%20a%20given%20line%20in%20a%20given%20day.ipynb), when launching online the dependencies are already installed.

## Pick a date

First, we pick the day we want to analyze

In [1]:
import datetime
from ipywidgets import DatePicker
from IPython.display import display

date_widget = DatePicker(description='Date:', value=datetime.date(2021,12,21))
display(date_widget)

DatePicker(value=datetime.date(2021, 12, 21), description='Date:')

## List rides on that day

Now we need to pick a ride to analyze, but because we don't haev GTFS data yet we can't pick based on GTFS identifiers

So, we get 10 rides which occured on that date and pick one of them

You can see the URL of the request being made to stride api

In [2]:
import stride
import pandas as pd

pd.DataFrame(stride.get('/siri_rides/list', {
    'limit': 10,
    'scheduled_start_time_from': datetime.datetime.combine(date_widget.value, datetime.time(), datetime.timezone.utc),
    'scheduled_start_time_to': datetime.datetime.combine(date_widget.value, datetime.time(23,59), datetime.timezone.utc),
}, pre_requests_callback='print'))

https://open-bus-stride-api.hasadna.org.il/siri_rides/list?limit=10&scheduled_start_time_from=2021-12-21T00%3A00%3A00.000000%2B0000&scheduled_start_time_to=2021-12-21T23%3A59%3A00.000000%2B0000


,id,siri_route_id,journey_ref,scheduled_start_time,vehicle_ref,updated_first_last_vehicle_locations,first_vehicle_location_id,last_vehicle_location_id,updated_duration_minutes,duration_minutes
0,3768012,3702,2021-12-21-2122158,2021-12-21T00:00:00+00:00,7660169,2021-12-21 02:00:15.344003+00:00,203882356,203883220,2021-12-21 08:00:05.699287+00:00,27
1,3767991,1834,2021-12-21-46511307,2021-12-21T00:00:00+00:00,7799969,2021-12-21 01:00:52.996683+00:00,203879174,203879174,2021-12-23 02:00:07.650709+00:00,0
2,3767988,3707,2021-12-21-11254277,2021-12-21T00:00:00+00:00,70154102,2021-12-21 01:00:52.957958+00:00,203879077,203881146,2021-12-21 07:00:07.484184+00:00,46
3,3767989,3661,2021-12-21-10390148,2021-12-21T00:00:00+00:00,7528869,2021-12-21 02:00:32.307424+00:00,203879102,203882071,2021-12-21 08:00:05.912945+00:00,73
4,3767990,3703,2021-12-21-2122670,2021-12-21T00:00:00+00:00,23309702,2021-12-21 02:00:32.310994+00:00,203879114,203881673,2021-12-21 08:00:05.915646+00:00,61
5,3767985,3654,2021-12-21-1534460,2021-12-21T00:00:00+00:00,7783569,2021-12-21 02:00:37.336469+00:00,203878861,203881670,2021-12-21 08:00:05.988634+00:00,65
6,3767984,1487,2021-12-21-36958754,2021-12-21T00:00:00+00:00,8323908,2021-12-21 01:00:58.119676+00:00,203878847,203880532,2021-12-21 07:00:07.886764+00:00,35
7,3767986,1719,2021-12-21-24910351,2021-12-21T00:00:00+00:00,63538801,2021-12-21 01:00:58.127514+00:00,203878864,203881497,2021-12-21 07:00:07.889753+00:00,60
8,3767987,3695,2021-12-21-11252999,2021-12-21T00:00:00+00:00,70139102,2021-12-21 01:00:58.131766+00:00,203878957,203881264,2021-12-21 07:00:07.893056+00:00,52
9,3767993,3634,2021-12-21-22383263,2021-12-21T00:15:00+00:00,7826769,2021-12-21 02:00:32.321122+00:00,203879714,203882067,2021-12-21 08:00:05.918515+00:00,62


## Pick a ride

Now we can choose an id from above list to analyze

In [3]:
from ipywidgets import Text

ride_id_widget = Text(
    description='Ride ID:',
    value='3767988'
)
display(ride_id_widget)

Text(value='3767988', description='Ride ID:')

## Get the ride-stops

To get relevant vehicle locations we need to get all the ride-stops on that ride

In [4]:
siri_ride_stops = pd.DataFrame(stride.iterate('/siri_ride_stops/list', {
    'siri_ride_ids': ride_id_widget.value,
}, pre_requests_callback='print'))
siri_ride_stops

https://open-bus-stride-api.hasadna.org.il/siri_ride_stops/list?siri_ride_ids=3767988&offset=0
https://open-bus-stride-api.hasadna.org.il/siri_ride_stops/list?siri_ride_ids=3767988&offset=35


,id,siri_stop_id,siri_ride_id,order
0,97719721,189,3767988,1
1,97719764,188,3767988,2
2,97719808,3909,3767988,3
3,97719828,7001,3767988,4
4,97719850,10196,3767988,6
5,97719894,11985,3767988,7
6,97719914,13953,3767988,9
7,97719952,14537,3767988,10
8,97719972,15568,3767988,11
9,97719987,16881,3767988,12


## Get the vehicle locations

Now we use the above list of ride-stop ids to get the list of vehicle locations

In [5]:
siri_vehicle_locations = pd.DataFrame(stride.iterate('/siri_vehicle_locations/list', {
    'siri_ride_stop_ids': ','.join(map(str, siri_ride_stops.id.to_list())),
    'order_by': 'distance_from_journey_start asc'
}, pre_requests_callback='print'))
siri_vehicle_locations

https://open-bus-stride-api.hasadna.org.il/siri_vehicle_locations/list?siri_ride_stop_ids=97719721%2C97719764%2C97719808%2C97719828%2C97719850%2C97719894%2C97719914%2C97719952%2C97719972%2C97719987%2C97720001%2C97720014%2C97720043%2C97720061%2C97720134%2C97720151%2C97720163%2C97720173%2C97720199%2C97720209%2C97720218%2C97720228%2C97720236%2C97720248%2C97720264%2C97720281%2C97720295%2C97720307%2C97720319%2C97720331%2C97720343%2C97720355%2C97720367%2C97720382%2C97720417&order_by=distance_from_journey_start+asc&offset=0
https://open-bus-stride-api.hasadna.org.il/siri_vehicle_locations/list?siri_ride_stop_ids=97719721%2C97719764%2C97719808%2C97719828%2C97719850%2C97719894%2C97719914%2C97719952%2C97719972%2C97719987%2C97720001%2C97720014%2C97720043%2C97720061%2C97720134%2C97720151%2C97720163%2C97720173%2C97720199%2C97720209%2C97720218%2C97720228%2C97720236%2C97720248%2C97720264%2C97720281%2C97720295%2C97720307%2C97720319%2C97720331%2C97720343%2C97720355%2C97720367%2C97720382%2C97720417&orde

,id,siri_snapshot_id,siri_ride_stop_id,recorded_at_time,lon,lat,bearing,velocity,distance_from_journey_start
0,203879077,168563,97719721,2021-12-20T23:59:33+00:00,35.090191,32.859379,31,0,0
1,203879139,168564,97719721,2021-12-21T00:00:33+00:00,35.090479,32.859177,202,26,0
2,203879201,168565,97719764,2021-12-21T00:01:35+00:00,35.093636,32.857899,89,35,459
3,203879262,168566,97719764,2021-12-21T00:02:36+00:00,35.094316,32.857355,185,35,559
4,203879321,168567,97719808,2021-12-21T00:03:37+00:00,35.093567,32.851214,186,74,1254
5,203879380,168568,97719828,2021-12-21T00:04:39+00:00,35.091681,32.846995,208,53,1748
6,203879438,168569,97719850,2021-12-21T00:05:41+00:00,35.088449,32.841953,210,22,2386
7,203879495,168570,97719850,2021-12-21T00:06:42+00:00,35.084736,32.836216,206,42,3125
8,203879550,168571,97719894,2021-12-21T00:07:43+00:00,35.080864,32.830151,208,55,3882
9,203879603,168572,97719914,2021-12-21T00:08:43+00:00,35.079628,32.824300,185,48,4563
